In [10]:
# Load environment variables from parent directory and set up auto-reload
import os

from dotenv import load_dotenv

load_dotenv(os.path.join("..", ".env"), override=True)

%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


## Context Isolation: Sub-agents

#TODO(Geoff): Can add more here. 

* Agent context can grow quickly. 
* Agents face several long context-related problems.
* A primary problem is context clash or confusion. 
* Context with mixed objectives can lead to suboptimal performance.
* [Context isolation](https://blog.langchain.com/context-engineering-for-agents/) is a good way to address these problems.
* We can delegate tasks to [specialized sub-agents](https://www.anthropic.com/engineering/multi-agent-research-system).
* Each sub-agent has its own context window.
* This prevents context clashes, confusion, poisoning, and dilution.

### Sub-agent delegation

* The primary insight is that we can create sub-agents with different tool sets.
* Each sub-agent will be stores in a dictionary (registry) with `subagent_type` as the key.
* Tasks can be delegated to sub-agents with a tool call, `task(description, subagent_type)`.
* The work done by the sub-agent is returned as a `ToolMessage` to the parent agent.

In [11]:
%%writefile ../src/deep_agents_from_scratch/task_tool.py

from typing import Annotated, NotRequired, TypedDict

from langchain_core.messages import ToolMessage
from langchain_core.tools import BaseTool, InjectedToolCallId, tool
from langgraph.prebuilt import InjectedState, create_react_agent
from langgraph.types import Command

from deep_agents_from_scratch.prompts import TASK_DESCRIPTION_PREFIX
from deep_agents_from_scratch.state import DeepAgentState

class SubAgent(TypedDict):
    """Configuration for a specialized sub-agent."""
    name: str
    description: str
    prompt: str
    tools: NotRequired[list[str]]

def _create_task_tool(tools, subagents: list[SubAgent], model, state_schema):
    """
    Create a task delegation tool that enables context isolation through sub-agents.
    
    This function implements the core pattern for spawning specialized sub-agents with
    isolated contexts, preventing context clash and confusion in complex multi-step tasks.
    
    Args:
        tools: List of available tools that can be assigned to sub-agents
        subagents: List of specialized sub-agent configurations
        model: The language model to use for all agents
        state_schema: The state schema (typically DeepAgentState)
    
    Returns:
        A 'task' tool that can delegate work to specialized sub-agents
    """

    # Create agent registry
    agents = {} 
    
    # Build tool name mapping for selective tool assignment
    tools_by_name = {}
    for tool_ in tools:
        if not isinstance(tool_, BaseTool):
            tool_ = tool(tool_)
        tools_by_name[tool_.name] = tool_
        
    # Create specialized sub-agents based on configurations
    for _agent in subagents:
        if "tools" in _agent:
            # Use specific tools if specified
            _tools = [tools_by_name[t] for t in _agent["tools"]]
        else:
            # Default to all tools
            _tools = tools
        agents[_agent["name"]] = create_react_agent(
            model, prompt=_agent["prompt"], tools=_tools, state_schema=state_schema
        )

    # Generate description of available sub-agents for the tool description
    other_agents_string = [
        f"- {_agent['name']}: {_agent['description']}" for _agent in subagents
    ]

    @tool(description=TASK_DESCRIPTION_PREFIX.format(other_agents=other_agents_string))
    def task(
        description: str,
        subagent_type: str,
        state: Annotated[DeepAgentState, InjectedState],
        tool_call_id: Annotated[str, InjectedToolCallId],
    ):
        """
        Delegate a task to a specialized sub-agent with isolated context.
        
        This creates a fresh context for the sub-agent containing only the task description,
        preventing context pollution from the parent agent's conversation history.
        """
        # Validate requested agent type exists
        if subagent_type not in agents:
            return f"Error: invoked agent of type {subagent_type}, the only allowed types are {[f'`{k}`' for k in agents]}"
        
        # Get the requested sub-agent
        sub_agent = agents[subagent_type]
        
        # Create isolated context with only the task description
        # This is the key to context isolation - no parent history
        state["messages"] = [{"role": "user", "content": description}]
        
        # Execute the sub-agent in isolation
        result = sub_agent.invoke(state)
        
        # Return results to parent agent via Command state update
        return Command(
            update={
                "files": result.get("files", {}),  # Merge any file changes
                "messages": [
                    # Sub-agent result becomes a ToolMessage in parent context
                    ToolMessage(
                        result["messages"][-1].content, tool_call_id=tool_call_id
                    )
                ],
            }
        )

    return task

Overwriting ../src/deep_agents_from_scratch/task_tool.py


Now, we can define specific sub-agents.

We allow the system to call them with the `task` tool.

#TODO(Geoff/RLM): Explain the prompting / strategy.

In [17]:
from utils import show_prompt

from deep_agents_from_scratch.prompts import AGENT_SYSTEM_PROMPT

show_prompt(AGENT_SYSTEM_PROMPT)

╭──────────────────────────────────────────────────── Prompt ─────────────────────────────────────────────────────╮
│                                                                                                                 │
│  You are a research supervisor with access to file management and task delegation capabilities. For context,    │
│  today's date is {date}.                                                                                        │
│                                                                                                                 │
│  <Task>                                                                                                         │
│  Your role is to coordinate research by delegating tasks to specialized sub-agents and managing research        │
│  artifacts through a virtual file system. You should delegate specific research tasks and organize findings     │
│  systematically.                                                                                                │
│  </Task>                                                                                                        │
│                                                                                                                 │
│  <Available Tools>                                                                                              │
│                                                                                                                 │
│  1. **task(description, subagent_type)**: Delegate research tasks to specialized sub-agents                     │
│     - description: Clear, specific research question or task                                                    │
│     - subagent_type: Type of agent to use (e.g., "research-agent")                                              │
│                                                                                                                 │
│  2. **ls()**: List files in your virtual workspace (no parameters needed)                                       │
│                                                                                                                 │
│  3. **read_file(file_path, offset=0, limit=2000)**: Read content from files                                     │
│     - file_path: Path to file you want to read                                                                  │
│     - offset: Starting line (optional, default 0)                                                               │
│     - limit: Max lines to read (optional, default 2000)                                                         │
│                                                                                                                 │
│  4. **write_file(file_path, content)**: ⚠️ BOTH PARAMETERS ALWAYS REQUIRED ⚠️                                     │
│     - file_path: Where to create the file (e.g., "research_question.md")                                        │
│     - content: The complete text content to write (NEVER SKIP THIS!)                                            │
│                                                                                                                 │
│  5. **write_todos(todos)**: Track research progress and plan next steps                                         │
│     - todos: List of todo items with status (pending/in_progress/completed)                                     │
│                                                                                                                 │
│  🔴 **AFTER EACH SUB-AGENT RESPONSE**: When you get research results from task tool, you MUST use               │
│  write_file(file_path, content) with BOTH parameters to save the findings. Do not just provide file_path        │
│  alone! 🔴                                                                                                      │
│                                                       

In [ ]:
from datetime import datetime

from IPython.display import Image, display
from langchain.chat_models import init_chat_model
from langchain_core.tools import tool
from langgraph.prebuilt import create_react_agent
from tavily import TavilyClient

from deep_agents_from_scratch.file_tools import ls, read_file, write_file
from deep_agents_from_scratch.prompts import (
    AGENT_SYSTEM_PROMPT,
    SIMPLE_RESEARCH_INSTRUCTIONS,
)
from deep_agents_from_scratch.state import DeepAgentState
from deep_agents_from_scratch.task_tool import _create_task_tool
from deep_agents_from_scratch.todo_tool import write_todos

# Limits 
max_concurrent_research_units = 3 
max_researcher_iterations = 3

@tool(parse_docstring=True)
def web_search(
    query: str,
):
    """Search the web for information on a specific topic.
    
    This tool uses Tavily to perform web searches and returns relevant results
    for the given query. Use this when you need to gather information from
    the internet about any topic.
    
    Args:
        query: The search query string. Be specific and clear about what 
               information you're looking for.
    
    Returns:
        Search results from Tavily including titles, URLs, and content snippets
        from relevant web pages.
        
    Example:
        web_search("machine learning applications in healthcare")
    """
    tavily_client = TavilyClient()
    search_results = tavily_client.search(
        query,
        max_results=5,
        include_raw_content=False,
        topic="general",
    )
    return search_results

# Create research sub-agent
research_sub_agent = {
    "name": "research-agent",
    "description": "Delegate research to the sub-agent researcher. Only give this researcher one topic at a time.",
    "prompt": SIMPLE_RESEARCH_INSTRUCTIONS,
    "tools": ["web_search"] 
}

# Create agent using create_react_agent directly
model = init_chat_model(model="anthropic:claude-sonnet-4-20250514", temperature=0.0)

# Tools for sub-agent
sub_agent_tools = [web_search]

# Create task tool to delegate tasks to sub-agents
task_tool = _create_task_tool(
    sub_agent_tools,
    [research_sub_agent],
    model,
    DeepAgentState
)

# Tools 
built_in_tools = [ls, read_file, write_file, write_todos]
delegation_tools = [task_tool] 

# Create agent with system prompt
agent = create_react_agent(
    model, 
    built_in_tools + delegation_tools, 
    prompt=AGENT_SYSTEM_PROMPT.format(
        max_concurrent_research_units=max_concurrent_research_units,
        max_researcher_iterations=max_researcher_iterations,
        date=datetime.now().strftime("%a %b %-d, %Y") 
    ), 
    state_schema=DeepAgentState
)

# Show the agent
display(Image(agent.get_graph(xray=True).draw_mermaid_png()))

In [ ]:
# Example usage
result = agent.invoke({
    "messages": [{"role": "user", "content": "Tell me about model context protocol (MCP) and how it works."}],
    "files": {}
})

#TODO(Geoff / Lance): Need a lot of guidance on the system:
#1/Smart TODO creation. It tends to create large # of TODO. Many become sub-agent tasks.
#2/Tasks: Each task spawns a sub-agents. Lots of token risk! 
#3/Files: Smart use of the fs.   

In [16]:
from utils import format_messages

format_messages(result['messages'])

╭─────────────────────────────────────────────────── 🧑 Human ────────────────────────────────────────────────────╮
│ Tell me about model context protocol (MCP) and how it works.                                                    │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭───────────────────────────────────────────────────── 📝 AI ─────────────────────────────────────────────────────╮
│                                                                                                                 │
│                                                                                                                 │
│ 🔧 Tool Call: write_file                                                                                        │
│    Args: {                                                                                                      │
│   "file_path": "research_question.md",                                                                          │
│   "content": "Title: Model Context Protocol (MCP) \u2013 What it is and how it works\n\nRequest date:           │
│ 2025-08-13\n\nUser question: \"Tell me about model context protocol (MCP) and how it works.\"\n\nScope and      │
│ clarifications to cover:\n- Who created/maintains MCP and its purpose\n- High-level overview and motivation     │
│ (provider-agnostic tool/resource access for LLMs)\n- Architecture: client vs. server, capabilities negotiation, │
│ message flow\n- Core concepts/primitives (e.g., tools/actions, resources, prompts/templates, filesystems,       │
│ memory)\n- Transport layer options supported by the spec\n- Security and permission model, isolation,           │
│ sandboxing\n- Example workflows (how a model calls a tool via MCP)\n- Developer ergonomics: how to build an MCP │
│ server; SDKs; example implementations\n- Ecosystem: official clients (e.g., editor integrations, desktop apps), │
│ notable servers, community adoption\n- Comparison to adjacent concepts (OpenAI function calling/tools,          │
│ LangChain tool abstractions), and when to use MCP\n- Limitations and considerations\n\nDeliverable: A concise   │
│ explanation to the user, with accurate, up-to-date details and concrete examples. Preserve full research notes  │
│ in findings_mcp.md."                                                                                            │
│ }                                                                                                               │
│    ID: call_2pqBWQNbhH4UI4ofHmWB7xiQ                                                                            │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭──────────────────────────────────────────────── 🔧 Tool Output ─────────────────────────────────────────────────╮
│ Updated file research_question.md                                                                               │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭───────────────────────────────────────────────────── 📝 AI ─────────────────────────────────────────────────────╮
│                                                                                                                 │
│                                                                                                                 │
│ 🔧 Tool Call: write_todos                                                                                       │
│    Args: {                                                                                                      │
│   "todos": [                                                                                                    │
│     {                                                                                                           │
│       "content": "Draft concise user-facing answer on MCP from existing knowledge while initiating targeted     │
│ fact-checking research.",                                                                                       │
│       "status": "in_progress"                                                                                   │
│     },                                                                                                          │
│     {                                                                                                           │
│       "content": "Verify current MCP ownership/maintainers and specification version; collect primary           │
│ sources.",                                                                                                      │
│       "status": "pending"                                                                                       │
│     },                                                                                                          │
│     {                                                                                                           │
│       "content": "Document architecture: client-server roles, capabilities, transports; gather diagrams or      │
│ descriptions from spec.",                                                                                       │
│       "status": "pending"                                                                                       │
│     },                                                                                                          │
│     {                                                                                                           │
│       "content": "Gather info on core primitives: tools/resources/prompts/files/file systems and how invocation │
│ works.",                                                                                                        │
│       "status": "pending"                                                                                       │
│     },                                                                                                          │
│     {                                                                                                           │
│       "content": "Collect examples and developer guidance: how to implement a server; SDKs; reference           │
│ servers.",                                                                                                      │
│       "status": "pending"                                                                                       │
│     },                                                                                                          │
│     {                                                                                                           │
│       "content": "Survey ecosystem: official clients (editor/desktop), notable community servers, adoption      │
│ examples.",                                                                                                     │
│       "status": "pending"                               

╭──────────────────────────────────────────────── 🔧 Tool Output ─────────────────────────────────────────────────╮
│ Updated todo list to [{'content': 'Draft concise user-facing answer on MCP from existing knowledge while        │
│ initiating targeted fact-checking research.', 'status': 'in_progress'}, {'content': 'Verify current MCP         │
│ ownership/maintainers and specification version; collect primary sources.', 'status': 'pending'}, {'content':   │
│ 'Document architecture: client-server roles, capabilities, transports; gather diagrams or descriptions from     │
│ spec.', 'status': 'pending'}, {'content': 'Gather info on core primitives: tools/resources/prompts/files/file   │
│ systems and how invocation works.', 'status': 'pending'}, {'content': 'Collect examples and developer guidance: │
│ how to implement a server; SDKs; reference servers.', 'status': 'pending'}, {'content': 'Survey ecosystem:      │
│ official clients (editor/desktop), notable community servers, adoption examples.', 'status': 'pending'},        │
│ {'content': 'Security model and permissions: what isolation and consent flows exist.', 'status': 'pending'},    │
│ {'content': 'Write findings into findings_mcp.md and prepare synthesis.md for final answer.', 'status':         │
│ 'pending'}]                                                                                                     │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

...

╭───────────────────────────────────────────────────── 📝 AI ─────────────────────────────────────────────────────╮
│                                                                                                                 │
│                                                                                                                 │
│ 🔧 Tool Call: ls                                                                                                │
│    Args: {}                                                                                                     │
│    ID: call_UNlHQRk39YusdKycTIITNDKK                                                                            │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭──────────────────────────────────────────────── 🔧 Tool Output ─────────────────────────────────────────────────╮
│ ["research_question.md", "research_plan.md", "findings_mcp_spec.md", "findings_mcp_dev.md",                     │
│ "findings_mcp_ecosystem.md"]                                                                                    │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭───────────────────────────────────────────────────── 📝 AI ─────────────────────────────────────────────────────╮
│                                                                                                                 │
│                                                                                                                 │
│ 🔧 Tool Call: write_file                                                                                        │
│    Args: {                                                                                                      │
│   "file_path": "synthesis.md",                                                                                  │
│   "content": "Model Context Protocol (MCP) \u2014 what it is and how it works\n\nHigh level\n- MCP is an open,  │
│ vendor-agnostic protocol for connecting AI assistants (clients) to external tools and data sources (servers).   │
│ It standardizes how assistants discover and call tools, fetch resources, and reuse prompts provided by          │
│ independent MCP servers. Think of it like the Language Server Protocol, but for AI tools and context.\n\nCore   │
│ ideas\n- Roles: A host app (chat UI, IDE, desktop client) runs an MCP client. External programs run as MCP      │
│ servers. The client connects to a server over a transport (stdio for local, or Streamable HTTP), discovers      │
│ capabilities, and invokes them.\n- Primitives:\n  - Tools: callable actions with JSON schemas for inputs;       │
│ invoked via tools/call\n  - Resources: readable data items the server exposes for context\n  - Prompts:         │
│ reusable prompt templates/snippets\n  - Discovery and sync happen via list methods (e.g., tools/list) and       │
│ list_changed notifications\n- Transport: JSON-RPC 2.0 over stdio or streamable HTTP (POST + optional SSE).      │
│ Legacy HTTP+SSE exists but is deprecated.\n- Security model: Host-centric. Clients must get explicit user       │
│ consent before exposing user data to servers, show tool inputs for review, and isolate servers (treat them as   │
│ untrusted processes) where possible.\n\nHow a call works (simplified)\n1) Connect and initialize: Client        │
│ connects to a server and initializes the session.\n2) Discover: Client calls tools/list (and resources/prompts  │
│ list) to fetch what the server offers.\n3) Consent: Host surfaces tools and intended inputs to the user for     │
│ approval.\n4) Invoke: Client sends tools/call with a tool name and arguments; server returns results (text or   │
│ structured content). Servers can also notify clients when capabilities change.\n\nWhy use MCP\n- Portability:   │
│ The same MCP server can be used by different assistants (e.g., Claude Desktop, IDE clients) without             │
│ re-implementing provider-specific function schemas.\n- Modularity: Keep tools and data connectors in standalone │
│ processes/services with their own auth and policies.\n- Ecosystem: Official SDKs exist for TypeScript, Python,  │
│ and Rust; the Inspector tool helps test any server.\n\nDeveloper snapshot\n- Build servers with the official    │
│ SDKs (@modelcontextprotocol/sdk for TypeScript; mcp on PyPI for Python). Expose tools/resources/prompts and     │
│ serve over stdio or HTTP. Use the Inspector (npx @modelcontextprotocol/inspector \u2026) to list capabilities   │
│ and call tools during development.\n\nEcosystem and usage\n- Clients: Claude Desktop supports connecting MCP    │
│ servers. JetBrains IDEs ship an MCP Server exposing IDE actions; Microsoft documents Dataverse as an MCP server │
│ usable from Claude Desktop and VS Code (Copilot) as clients.\n- Community: Dozens of servers exist for          │
│ filesystems, Git, databases, Slack, Notion, search, and more\u2014see curated \u201cawesome MCP servers\u201d   │
│ lists.\n\nLimitations and considerations\n- Not every assistant supports MCP yet; setup can require proxies or  │
│ configuration. Because MCP can expose powerful actions, 

╭──────────────────────────────────────────────── 🔧 Tool Output ─────────────────────────────────────────────────╮
│ Updated file synthesis.md                                                                                       │
╰─────────────────────────────────────────────────────────────────────────────────────────────────────────────────╯

╭───────────────────────────────────────────────────── 📝 AI ─────────────────────────────────────────────────────╮
│ Here’s a concise overview of the Model Context Protocol (MCP) and how it works.                                 │
│                                                                                                                 │
│ What it is                                                                                                      │
│ - MCP is an open, vendor-agnostic protocol that lets AI assistants connect to external tools and data sources   │
│ safely and consistently.                                                                                        │
│ - It’s inspired by the Language Server Protocol: assistants act as clients; integrations run as servers that    │
│ expose capabilities the assistant can discover and use.                                                         │
│                                                                                                                 │
│ Core architecture                                                                                               │
│ - Roles                                                                                                         │
│   - Client (inside a host app like a chat UI, IDE, or desktop app) connects to a server and manages user        │
│ consent.                                                                                                        │
│   - Server (a separate process/service) exposes capabilities to the client.                                     │
│ - Protocol                                                                                                      │
│   - JSON-RPC 2.0 messages for requests, responses, and notifications.                                           │
│   - Discovery happens via list methods; real-time updates use “list_changed” notifications.                     │
│ - Primitives                                                                                                    │
│   - Tools: callable actions with JSON-schema’d inputs (discovered via tools/list, invoked via tools/call).      │
│   - Resources: readable items (files, URLs, DB rows, etc.) that a server exposes for context.                   │
│   - Prompts: reusable prompt templates/snippets.                                                                │
│ - Transports                                                                                                    │
│   - Local: stdio (common for local processes).                                                                  │
│   - Network: “streamable HTTP” (HTTP POSTs client→server with optional Server-Sent Events stream                │
│ server→client).                                                                                                 │
│   - Legacy HTTP+SSE exists but is deprecated.                                                                   │
│                                                                                                                 │
│ Security model                                                                                                  │
│ - Host-centric: the client/host obtains explicit user consent before sharing user data or calling sensitive     │
│ tools.                                                                                                          │
│ - Isolation: treat servers as untrusted by default; run them in separate processes/containers and apply         │
│ least-privilege.                                                                                                │
│ - Visibility: surface tool inputs/intent to the user before invocation where possible; audit/log important      │
│ actions.                                                                                                        │
│                                                        